In [27]:
import numpy as np
import matplotlib.pyplot as plt
import struct
from sklearn import svm, metrics
import math
%matplotlib inline

In [5]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [14]:
## loading mnist dataset

raw_train = read_idx("./data/train-patterns-idx3-ubyte")
train_data = np.reshape(raw_train, (60000, 28*28))
train_label = read_idx("./data/train-labels-idx1-ubyte")

raw_test = read_idx("./data/mnist_new_test-patterns-idx3-ubyte")
test_data = np.reshape(raw_test, (10000, 28*28))
test_label = read_idx("./data/mnist_new_test-labels-idx1-ubyte")

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


In [15]:
## Standardizing the features
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(train_data)
train_std = sc.transform(train_data)
test_std = sc.transform(test_data)

In [16]:
print(train_std.shape, train_label.shape)

(60000, 784) (60000,)


In [17]:
print(test_std.shape, test_label.shape)

(10000, 784) (10000,)


In [32]:
class SVM(object):
    def __init__(self, C=0.1, eta=0.001, batch_size=128, epochs=25, class_num=0, shuffle=True):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.class_num = class_num
        self.shuffle = shuffle
    
    def fit(self, X, y):
        X_n, X_f = np.shape(X) # X_n = 60000(data pts), X_f = 784(features)
        self.class_num = len(np.unique(y)) # class_num = 10
        
        # OvR
        y_ovr = self.ovr(y)
        
        ## initialize w, b
        self.w = np.random.randn(X_f, self.class_num) # (784,10)
        self.b = np.random.randn(1, self.class_num)
        
        for epoch in range(self.epochs):
            if self.shuffle:
                X_shuffled, y_shuffled = self.shuffles(X, y_ovr)
            
            # batch_count = dataset_size / batch_size
            batch_count = math.ceil(X.shape[0] / self.batch_size)
            for a in range(batch_count):
                X_batch = X[self.batch_size*a:self.batch_size*(a+1)] # X_batch[batch_size][784]
                y_batch = y[self.batch_size*a:self.batch_size*(a+1)] # y_batch[batch_size][784]
                
                sum_w = np.zeros(X_f)
                sum_b = 0.0
                
                
#                for Xi, yi in zip(X_batch, y_batch):
#                    loss = self.hinge_loss(Xi, yi, self.w, self.b) # loss[batch_size][class_num]
#                    if loss < 1: # yi*(np.dot(Xi, self.w) + self.b)
#                        sum_w += (-yi*Xi)
#                        sum_b += (-yi)
                loss = self.hinge_loss(X_batch, y_batch)
                if loss < 1:
                    sum_w += (-y_batch*X_batch)
                    sum_b += (-y_batch)
                self.w = self.w - (self.eta * ((sum_w/batch_size)+(1/self.C)*self.w))
                self.b = self.b - self.eta * sum_b / batch_size
    
    def ovr(self, y):
        y_ovr = np.ones((np.shape(y)[0], self.class_num)) # y_ovr[60000][class_num]
        
        for i in range(self.class_num):
            y_ovr[:, i][y != i] = -1
        return y_ovr
    
    def shuffles(self, X, y):
        ran = np.arange(0, np.shape(X)[0])
        np.random.shuffle(ran)
        return X[ran], y[ran]
    
    def hinge_loss(self, X, y):
        loss = y*(np.dot(X, self.w) + self.b) # yi * (Wi*Xi+b)
        return loss
    
    def predict(self, X):
        score = np.dot(X, self.w) + self.b
        pred = np.argmax(class_score, axis=1)
        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        sc = np.mean(pred == y)
        
        return score

In [33]:
svm = SVM()
result = svm.fit(train_std, train_label)

ValueError: operands could not be broadcast together with shapes (128,) (128,10) 